In [1]:
%cd ../

/mnt/data/ai4earthquake/zhangtianning/projects/AI4Q


# tb2wandb

In [2]:
import os


import wandb

import json
from mltool.loggingsystem import LoggingSystem
from config.utils import print_namespace_tree, convert_namespace_tree
import h5py
import pandas as pd
import tabulate
import math

from config.utils import print_namespace_tree, convert_namespace_tree, flatten_dict, retrieve_dict, build_kv_list
from project_arguements import ProjectConfig, get_args, get_parallel_config_of_accelerator, to_dict,save

In [3]:
def create_logsys(args,save_config=True):
    local_rank = 0
    SAVE_PATH  = args.output_dir
    recorder_list = ['wandb_runtime']
    use_wandb  = 'wandb_runtime' if 'wandb' in recorder_list else False
    logsys   = LoggingSystem(local_rank==0,SAVE_PATH,seed=args.task.seed,
                             use_wandb=use_wandb, recorder_list=recorder_list, flag='train',
                             disable_progress_bar=False)
    hparam_dict={}
    metric_dict={}
    dirname          = SAVE_PATH
    dirname,name     = os.path.split(dirname)
    _ = logsys.create_recorder(hparam_dict={},metric_dict={},
                                args    = retrieve_dict(to_dict(args)),
                                project = f"{args.dataset_name}-{args.model.Predictor.prediction_type}",
                                entity  = "szztn951357",
                                group   = args.model_name,
                                job_type= None,
                                name    = name
                               ) 
   
    #if local_rank == 0:print_namespace_tree(args)
    return logsys

In [ ]:
!cat 

In [11]:
from tbparse import SummaryReader
from project_arguements import get_args
import wandb,json,os
trial_path = "checkpoints/instance.group.HH.hdf5/Goldfish.40M_A.GroupE2E/02_14_15_10-seed_20130901-d3e8/instance.group.HH.hdf5-recurrent/"
ROOTDIR    = os.path.dirname(trial_path.strip('/'))
args = get_args(os.path.join(ROOTDIR,'train_config.json'),args=['--task','train','--Resource','InstanceGroup'])
args.output_dir = ROOTDIR

In [12]:
import argparse
import re
import wandb 
from tbparse import SummaryReader
import os

In [16]:
from pathlib import Path

[PosixPath('checkpoints/instance.group.HH.hdf5/Goldfish.40M_A.GroupE2E/02_14_15_10-seed_20130901-d3e8/instance.group.HH.hdf5-recurrent/events.out.tfevents.1707894657.dlcyfnll2dt2xdos-master-0.92.0')]

In [13]:
summary_dir = trial_path
outdf = []
for filename in os.listdir(summary_dir):
    if 'events' not in filename:continue
    log_dir = os.path.join(summary_dir,filename)
    print(log_dir)
    reader = SummaryReader(log_dir)
    print
    df = reader.scalars

    if len(df) < 1: 
        print(f"==>no scalars at,pass")
        continue
    print("===>start parsing tensorboard..............")
    outdf.append(df)
assert len(outdf)==1
df = outdf[0]

checkpoints/instance.group.HH.hdf5/Goldfish.40M_A.GroupE2E/02_14_15_10-seed_20130901-d3e8/instance.group.HH.hdf5-recurrent/events.out.tfevents.1707894657.dlcyfnll2dt2xdos-master-0.92.0
===>start parsing tensorboard..............


In [14]:
df

,step,tag,value
0,204,epoch,1.000000
1,204,epoch,1.000000
2,408,epoch,2.000000
3,408,epoch,2.000000
4,612,epoch,3.000000
...,...,...,...
8845,9384,validation/loss_gro,22.817457
8846,9588,validation/loss_gro,22.728607
8847,9792,validation/loss_gro,22.828457
8848,9996,validation/loss_gro,22.780415


In [15]:
logsys = create_logsys(args, save_config=False)
all_pool={}
for step, tag, val in df.values:
    if step not in all_pool:all_pool[step]={}
    all_pool[step][tag]  =val 

for step, record in all_pool.items():
    record['step'] = step
    wandb.log(record,step =step)

logsys.close()

wandb: Currently logged in as: tianning (ai4quake). Use `wandb login --relogin` to force relogin


log at checkpoints/instance.group.HH.hdf5/Goldfish.40M_A.GroupE2E/02_14_15_10-seed_20130901-d3e8
key=strategy_name conflict. Can be ['task.sampling_strategy.train_sampling_strategy.strategy_name', 'task.sampling_strategy.valid_sampling_strategy.strategy_name', 'task.sampling_strategy.test_sampling_strategy.strategy_name']
So far it should be the strategy_name/early_warning, we will use the last two:train_sampling_strategy.strategy_name
So far it should be the strategy_name/early_warning, we will use the last two:valid_sampling_strategy.strategy_name
So far it should be the strategy_name/early_warning, we will use the last two:test_sampling_strategy.strategy_name
key=early_warning conflict. Can be ['task.sampling_strategy.train_sampling_strategy.early_warning', 'task.sampling_strategy.valid_sampling_strategy.early_warning', 'task.sampling_strategy.test_sampling_strategy.early_warning']
So far it should be the strategy_name/early_warning, we will use the last two:train_sampling_strategy.

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▂▃▃▅▆▆▇████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
validation/e_group_vector,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/loss_gro,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50.0
step,10200
train/loss,22.67693
train/lr,1e-05


# modify wandb

In [2]:
import wandb
import numpy as np
import re
api = wandb.Api()
runs= api.runs('ai4quake/stead.trace.BDLEELSSO-recurrent')

In [9]:
nickname_map = {
    "RetNetDecay_SignalSeaL1.512.1024.256.16.4": 'Goldfish.10M.L1',
    "RetNetDecay_SignalSeaL1.512.1024.256.16.8": 'Goldfish.20M.L1',

    "RetNetDecay_SignalSeaL1.3.512.1024.256.8": 'Goldfish.20M.L1',
    'RetNetDecay_SignalSeaL1.3.512.1024.256.4': 'Goldfish.10M.L1',

    'RetNetDecay_SignalSea.3.512.1024.256.1': 'Goldfish.2M.Sea',
    'RetNetDecay_SignalSea.512.1024.256.16.1': 'Goldfish.2M.Sea',
    'RetNetDecay_SignalSea.3.512.1024.256.4': 'Goldfish.10M.Sea',
    'RetNetDecay_SignalSea.512.1024.256.16.4': 'Goldfish.10M.Sea',

    'RetNetDecay_SignalSea.3.768.512.256.1': 'Goldfish.2M_Z.Sea',
    'RetNetDecay_SignalSeaL1.3.768.512.256.4': 'Goldfish.10M_Z.L1',
    'RetNetDecay_SignalSea.3.768.512.256.4': 'Goldfish.10M_Z.Sea',
    "RetNetDecay_SignalSea.768.512.256.16.4":'Goldfish.10M_Z.Sea',
    "RetNetDecay_SignalSea.768.512.256.16.1":'Goldfish.2M_Z.Sea',
    "RetNetDecay_SignalSeaL1.768.512.256.16.4":'Goldfish.10M_Z.L1',

    "RetNetDecay_ReLU.3.512.1024.256.4":'Goldfish.ReLU.10M',
    "RetNetDecay_NoBias.3.512.1024.256.4":'Goldbird.10M',
    "Goldfish.85M_B.L1":"Goldfish.85M_B.L1",
    "Goldfish.10M.L1":"Goldfish.10M.L1",
    "Goldfish.10M.Sea":"Goldfish.10M.Sea",
    'FlatfishS10M':'Flatfish.10M.Sea',
    'Flatfish.10M.A':'Flatfish.10M_A',
    'Flatfish.10M.Sea':'Flatfish.10M.Sea',
    'Goldfish.85M_B.Sea':'Goldfish.85M_B.Sea',
    'Flatfish.10M.L1':'Flatfish.10M.L1',
    'Sea': "Goldfish.10M.Sea",
    'L1': "Goldfish.10M.L1",
    
}

In [3]:
# groups=[]
# key = 'task.sampling_strategy.train_sampling_strategy.strategy_name'
# for run in runs:
#     if key in run.config and run.config[key] is not None:
#         print(run.config[key])
#         continue
#     if run.config['warning_window'] == -4:
#         run.config[key] = 'random_sample_in_ahead_L_to_p'
#         run.update()
#     else:
#         print(run.config['warning_window'])

In [4]:
for run in runs:
    if run.state == 'running':continue
    if 'use_confidence' not in run.config:
        run.config['use_confidence']='whole_sequence'
        run.update()
    if run.config['use_confidence'] is None:
        run.config['use_confidence']='whole_sequence'
        run.update()

In [10]:
groups=[]
for run in runs:
    if run.state == 'running':continue
    if run.config['nickname']=='Sea':
        run.config['nickname'] = "Goldfish.10M.Sea"
        run.update()

In [5]:
groups=[]
for run in runs:
    if run.state == 'running':continue

    if 'nickname' in run.config and run.config['nickname'] is not None and run.config['nickname'] != "":
        #print(run.config['nickname'])
        continue
    if run.group not in nickname_map:
        print(run.group,run.name)
        continue
    run.config['nickname'] = nickname_map[run.group]
    run.update()

In [6]:
for run in runs:
    if run.state == 'running':continue
    if 'report/time_base_p(second)' in run.summary or 'report_status/relaxtion(second)' in run.summary:
        if 'train_or_infer' not in run.config or run.config['train_or_infer'] == 'train':
            run.config['train_or_infer'] = 'infer'
            run.update()
            print(run.id)
    else:
        if 'train_or_infer' not in run.config:
            run.config['train_or_infer'] = 'train'
            run.update()
            print(run.id)

xo3j3sxs
rohh7eug
8afydj0y
q3txq751
nt2xmn5a
eis31e9a
pnmt2xk6
jp1czm31
b9s6u1kx
7ovuangk
e44sjlir
uig31s8i
b9cyh88d
k7cgzw2c
4y4huz42
3rbymwie
zzh1a12d
w3q4ibll
m5x0qbwx


In [8]:
# had_inference = {}
# for run in runs:
#     if run.state == 'running':continue
#     if ('use_confidence' in run.config and run.config['use_confidence'] == 'status'):
#         name = run.name
#         if name not in had_inference:
#             had_inference[name] = [False, run.config['output_dir']]
#         if 'report/time_base_p(second)' in run.summary:
#             had_inference[name][0] = True
# for key, (had_infer, path) in had_inference.items():
#     if not had_infer:
#         print(path)

checkpoints/stead.trace.BDLEELSSO_sample6000/Goldfish.85M_B.Sea/11_18_20_49-seed_21
checkpoints/stead.trace.BDLEELSSO_sample6000/Goldfish.85M_B.Sea/11_17_02_29-seed_21
checkpoints/stead.trace.BDLEELSSO_sample6000/RetNetDecay_SignalSea.3.768.512.256.1/first_layer_zero_space
checkpoints/stead.trace.BDLEELSSO_sample3000/RetNetDecay_SignalSea.3.768.512.256.4/11_03_11_42-seed_21


In [10]:
run.summary

{'_wandb': {'runtime': 10550}, '_timestamp': 1698681408.9736748, 'train/loss': 0.012031388595114632, 'itering/data': 0.016445517539978027, 'itering/model': 0.40226638317108154, 'validation/loss_sta': 0.16447711813038793, 'epoch': 100, 'itering/a_status': 0.9917168021202089, 'itering/loss_sta': 0.0164794921875, 'validation/a_status': 0.9674846653280588, '_runtime': 10555.143421888351, 'validation/loss': 0.16447711813038793, 'train/lr': 1.0244252618962858e-05, 'itering/loss': 0.012082751520079187, '_step': 22900}

In [8]:
import os
name_pool  = {}
for run in runs:
    if run.state == 'running':continue
    if 'downstream_task' not in run.config:
        continue
    if run.config['downstream_task'] != 'mse_xyS':continue
    ouput_dir = run.config['output_dir']
    if ouput_dir not in name_pool:name_pool[ouput_dir]=0
    if run.config['train_or_infer'] != 'train':
        if 'report/time_base_p(second)' not in run.summary:continue
        #if 'max_inference_length' not in name_pool[ouput_dir]:name_pool[ouput_dir]['max_inference_length'] = 0
        name_pool[ouput_dir] = max(run.summary['report/time_base_p(second)'],name_pool[ouput_dir] )
for ouput_dir, max_inference_length in name_pool.items():
    if max_inference_length<60:
        if os.path.exists(os.path.join(ouput_dir,'pytorch_model.bin')):
            pass
            print(ouput_dir,"\\")
        else:
            pass
            #print(p['train'][0])

checkpoints/stead.trace.BDLEELSSO_sample6000/Sea/11_30_10_52-seed_21 \
checkpoints/stead.trace.BDLEELSSO_sample6000/Goldfish.10M.ReLU.Sea/11_30_10_52-seed_21 \


In [4]:
import os
name_pool  = {}
for run in runs:
    if run.state == 'running':continue
    if 'downstream_task' not in run.config:
        raise
    if run.config['downstream_task'] == 'status':continue
    if run.name not in name_pool:name_pool[run.name]={}
    if run.config['train_or_infer'] not in name_pool[run.name]:name_pool[run.name][run.config['train_or_infer']]=[]
    name_pool[run.name][run.config['train_or_infer']].append(run.config['output_dir'])

for name, p in name_pool.items():
    if 'train' not in p:continue
    if len(p) <2:
        if os.path.exists(p['train'][0]):
            pass
            #print(p['train'][0])
        else:
            pass
            print(p['train'][0])

In [5]:
training_sampling_range_pool = {}
for run in runs:
    if run.state == 'running':continue
    name = run.name
    if run.config['train_or_infer'] == 'train':
        if 'train_sampling_strategy.strategy_name' in run.config and run.config['train_sampling_strategy.strategy_name']:
            if run.config['train_sampling_strategy.strategy_name'] in ['random_sample_in_ahead_L_to_p']:
                training_sampling_range = (-run.config['max_length'], 0, run.config['max_length'])
            elif run.config['train_sampling_strategy.strategy_name'] in ['random_sample_before_p_in_warning_window']:
                assert run.config['train_sampling_strategy.early_warning'] is not None
                training_sampling_range = (-run.config['train_sampling_strategy.early_warning'], 0, run.config['max_length'])
            else:
                print(run.config['train_sampling_strategy.strategy_name'])
                continue
        else:
            if 'warning_window' in run.config and run.config['warning_window']:
                if run.config['warning_window'] == -4:
                    start_sampling_pos_base_on_p = -run.config['max_length']
                    end_smapling_pos_base_on_p   = 0
                    training_sampling_range = (start_sampling_pos_base_on_p, end_smapling_pos_base_on_p,run.config['max_length'])
                    
            else:
                continue
        training_sampling_range_pool[run.name] = training_sampling_range

In [6]:
for run in runs:
    if run.state == 'running':continue
    if 'train_on' in run.config and run.config['train_on']:continue
    if run.name in training_sampling_range_pool:
        a = training_sampling_range_pool[run.name]
        run.config['train_on'] = f"({a[0]}, {a[1]}, {a[2]})"
        run.update()
        print(run.name)

In [5]:
for run in runs:
    if run.state == 'running':continue
    if 'train_on' in run.config:
        
        a = run.config['train_on'].strip('()').split(',')
        if a[1][0] == " ":
            continue
        
        #if a[0] == '(' and a[-1] == ')':continue
        run.config['train_on'] = f"({a[0]}, {a[1]}, {a[2]})"
        run.update()


In [31]:
def findkey(path,key = 'parallel_config.num_processes'):
    with open(path,'r') as f:
        for line in f:
            # Check if the line starts with the key
            if line.startswith(key):
                # Split the line at the colon and strip whitespace to get the value
                value = line.split(':')[1].strip()
                return value

In [9]:
import os
import glob
true_train_batchsize_pool = {}
for run in runs:
    if run.state == 'running':continue
    name = run.name

    if run.config['train_or_infer'] == 'train':
        if 'num_processes' not in run.config:
            if 'parallel_config' in run.config:
                num_processes =run.config['parallel_config']['num_processes']
            else:
                if 'output_dir' not in run.config:continue
                project_dir = run.config['output_dir']
                if os.path.exists(project_dir):
                    matching_paths = glob.glob(f'{project_dir}/**/hparams.yml', recursive=True)
                    if len(matching_paths)>1:
                        print(f"multiple hparam files, check {project_dir}")
                        continue
                    matching_paths=matching_paths[0]
                    num_processes = int(findkey(matching_paths))
                    if num_processes == 1:
                        print(run.config['output_dir'],num_processes)
                        continue

                else:
                    if name in ['11_15_12_43-seed_21', '11_15_21_47-seed_21']:
                        num_processes = 8
                    else:
                        print(f"not in this server: {run.config['output_dir']}")
                        continue
        else:
            num_processes = run.config['num_processes']
        true_batchsize = num_processes * run.config['batch_size'] * run.config['gradient_accumulation_steps']
        true_train_batchsize_pool[run.name] = true_batchsize

for run in runs:
    if run.state == 'running':continue
    if 'true_batchsize' in run.config:continue
    if run.name in true_train_batchsize_pool:
        run.config['true_batchsize'] = true_train_batchsize_pool[run.name]
        run.update()
        print(run.id)

NameError: name 'glob' is not defined

In [51]:
for run in runs:
    if run.state == 'running':continue
    if 'true_batchsize' in run.config:
        if 'trained_batch_size' in run.config:continue
        run.config['trained_batch_size'] = run.config['true_batchsize']
        run.update()

In [6]:
for run in runs:
    if run.state == 'running':continue
    if run.config['train_or_infer'] == 'train':continue
    if run.name in training_sampling_range_pool:
        run.config['train_on'] = training_sampling_range_pool[run.name]
        run.update()

In [7]:
import json
with open('checkpoints/stead.trace.BDLEELSSO_sample6000/RetNetDecay_SignalSea.3.512.1024.256.4/11_02_12_42-seed_21/train_config.json','r') as f:
    config = json.load(f)

In [18]:
run1_config = {}
for key,val in run2.config.items():
    if key not in config:
        run1_config[key] = val
    else:
        run1_config[key] = config[key]

In [19]:
run1_config['use_confidence']

'status'

In [20]:
run1.config = run1_config
run1.update()

# Add Sweep Parameter

In [2]:
import wandb
import numpy as np
import re
api = wandb.Api()
runs= api.runs('ai4quake/diting.tiny.trace.hdf5-recurrent')

In [3]:
for run in runs:
    if run.state == 'running':continue
    break

In [4]:
run.summary

{'itering/e_y': 28, 'itering/loss_y': 1200, '_runtime': 28.196041584014893, 'itering/loss': 3493.3333333333335, 'itering/e_shift': 35, '_step': 24, '_wandb': {'runtime': 23}, 'itering/e_x': 16.875, 'itering/a_status': 0.6254392266273499, 'itering/e_distance': 34.25, 'itering/loss_scilence': 1.625, '_timestamp': 1702101304.5870705, 'itering/data': 0.0008623600006103516, 'itering/model': 0.24268198013305664, 'itering/loss_x': 444, 'itering/loss_sta': 0.984375, 'itering/loss_active': 0.0830078125}